In [10]:

import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import os
import PIL

from imagecorrection.Airlight import Airlight
from imagecorrection.BoundCon import BoundCon
from imagecorrection.CalTransmission import CalTransmission
from imagecorrection.removeHaze import removeHaze
import matplotlib.pyplot as plt
import glob

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model
import pandas as pd
from PIL import Image, ImageEnhance
# from IPython.display import display


In [11]:

labels = open(r'..\yolo-coco-data\coco.names').read().strip().split('\n')
weights_path = r'..\yolo-coco-data\yolov4.weights'
configuration_path = r'..\yolo-coco-data\yolov4.cfg'
probability_minimum = 0.5
threshold = 0.3


In [12]:
network = cv2.dnn.readNetFromDarknet(configuration_path, weights_path)
layers_names_all = network.getLayerNames()


In [13]:
layers_names_output = [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]  # list 
print(layers_names_output)

['yolo_139', 'yolo_150', 'yolo_161']


In [14]:
def detect_model(image_input):
    blob = cv2.dnn.blobFromImage(image_input, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    # Check point
    blob_to_show = blob[0, :, :, :].transpose(1, 2, 0)
    network.setInput(blob)  # setting blob as input to the network
    output_from_network = network.forward(layers_names_output)
    np.random.seed(42)
    colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')
    bounding_boxes = []
    confidences = []
    class_numbers = []
    image_input_shape = image_input.shape
    h, w = image_input_shape[:2]  # Slicing from tuple only first two elements
    for result in output_from_network:
        for detection in result:
            scores = detection[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]
            if confidence_current > probability_minimum:
                box_current = detection[0:4] * np.array([w, h, w, h])
                x_center, y_center, box_width, box_height = box_current.astype('int')
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)
    pred_scores = []
    mean_avg_prec = 0
    obj_mean_avg = 0
    if len(results) > 0:
        for i in results.flatten():
            pred_scores.append(confidences[i])
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            colour_box_current = [int(j) for j in colours[class_numbers[i]]]
            cv2.rectangle(image_input, (x_min, y_min), (x_min + box_width, y_min + box_height),colour_box_current, 3) 
    if len(pred_scores) > 0:
        mean_avg_prec = sum(pred_scores)/len(results)
        obj_mean_avg = len(results) * mean_avg_prec
    return image_input, mean_avg_prec, "detected objects: {}".format(len(results)), "No of object detected * Mean Average precision : {}".format(obj_mean_avg)


In [15]:
def isbright(image, dim=50, thresh=0.5):
    # Resize image to 50x50
    image = cv2.resize(image, (dim, dim))
    # Convert color space to LAB format and extract L channel
    L, A, B = cv2.split(cv2.cvtColor(image, cv2.COLOR_BGR2LAB))
    # Normalize L channel by dividing all pixel values with maximum pixel value
    L = L/np.max(L)
    # Return True if mean is greater than thresh else False
    return np.mean(L) > thresh


In [9]:
# load image from disk
overall_mean_p = []
main_dir = r'..\images-for-testing'
for filename in os.listdir(main_dir):
    image_path = r'{}\{}'.format(main_dir, filename)
    file_name = image_path.split('\\')[-1]
    image_input = plt.imread(image_path)

    # find if image is bright or dark
    select_best_img = {}

    plt.figure(figsize=(15.0,15.0))
    orginal = detect_model(image_input)
    dect = orginal[0]
    select_best_img["orginal"] = orginal[3]
    im = Image.open(image_path)

    enhancer = ImageEnhance.Brightness(im)
    factor = 0.5 #darkens the image
    im_output_d = enhancer.enhance(factor)
    darken = detect_model(np.array(im_output_d))
    dect_d = darken[0]
    select_best_img["darken"] = darken[3]
      

    enhancer = ImageEnhance.Brightness(im)
    factor = 1.5 #brightens the image
    im_output_b = enhancer.enhance(factor)
    bright = detect_model(np.array(im_output_b))
    dect_b = bright[0]
    select_best_img["bright"] = bright[3]
       

    enhancer = ImageEnhance.Sharpness(im)
    factor = 2
    img_shrp = enhancer.enhance(factor)
    sharp = detect_model(np.array(img_shrp))
    dect_s = sharp[0]
    select_best_img["sharpness"] = sharp[3]
    

    enhancer = ImageEnhance.Contrast(im)
    factor = 1.5 #increase contrast
    im_output_c = enhancer.enhance(factor)
    contrast = detect_model(np.array(im_output_c))
    dect_c = contrast[0]
    select_best_img["contrast"] = contrast[3]
 

    enhancer = ImageEnhance.Sharpness(im_output_d)
    factor = 2
    img_dshrp = enhancer.enhance(factor)
    dark_sharp = detect_model(np.array(img_dshrp))
    dect_ds = dark_sharp[0]
    select_best_img["dark_sharp"] = dark_sharp[3]

    enhancer = ImageEnhance.Brightness(im_output_d)
    factor = 1.5 #brightens the image
    im_output_b = enhancer.enhance(factor)
    dark_bright = detect_model(np.array(im_output_b))
    dect_b =dark_bright[0]
    select_best_img["dark_bright"] = dark_bright[3]

    enhancer = ImageEnhance.Sharpness(im_output_b)
    factor = 2
    img_bshrp = enhancer.enhance(factor)
    bright_sharp = detect_model(np.array(img_bshrp))
    dect_bs = bright_sharp[0]
    select_best_img["bright_sharp"] = bright_sharp[3]
      
    enhancer = ImageEnhance.Sharpness(im_output_c)
    factor = 2
    img_cshrp = enhancer.enhance(factor)
    const_sharp = detect_model(np.array(img_cshrp))
    dect_cs = const_sharp[0]
    select_best_img["contrast_sharp"] = const_sharp[3]
  
    enhancer = ImageEnhance.Contrast(im_output_b)
    factor = 1.5 #increase contrast
    im_output_bc = enhancer.enhance(factor)
    bright_const = detect_model(np.array(im_output_bc))
    dect_bc = bright_const[0]
    select_best_img["bright_contrast"] = bright_const[3]

    enhancer = ImageEnhance.Color(im)
    factor = 1.5
    im_output_color = enhancer.enhance(factor)
    color = detect_model(np.array(im_output_color))
    dect_color = contrast[0]
    select_best_img["color"] = color[3]
   
      
    # Estimate Airlight
    windowSze = 15
    AirlightMethod = 'fast'
    A = Airlight(image_input, AirlightMethod, windowSze)

    # Calculate Boundary Constraints
    windowSze = 3
    C0 = 20         # Default value = 20 (as recommended in the paper)
    C1 = 300        # Default value = 300 (as recommended in the paper)
    Transmission = BoundCon(image_input, A, C0, C1, windowSze)                  #   Computing the Transmission using equation (7) in the paper

    # Refine estimate of transmission
    regularize_lambda = 1       # Default value = 1 (as recommended in the paper) --> Regularization parameter, the more this  value, the closer to the original patch wise transmission
    sigma = 0.7
    Transmission = CalTransmission(image_input, Transmission, regularize_lambda, sigma)     # Using contextual information

    # Perform DeHazing
    HazeCorrectedImg = removeHaze(image_input, Transmission, A, 0.85)
    dehaze = detect_model(HazeCorrectedImg)
    dect1 = dehaze[0]
    select_best_img["dehaze"] = dehaze[3]


    better_dect = max(select_best_img, key=select_best_img.get)
    best_img_path = r"C:\Users\Sai Krishna\Desktop\input\results\{}".format(file_name)
    if not better_dect:
        print('no object is dected in given image')
    elif better_dect == 'orginal':
        overall_mean_p.append(orginal[1])
        # plt.imsave(best_img_path, orginal[0])
    elif better_dect == 'darken':
        overall_mean_p.append(darken[1])
        # plt.imsave(best_img_path, darken[0])
    elif better_dect == 'bright':
        overall_mean_p.append(bright[1])
        # plt.imsave(best_img_path, bright[0])
    elif better_dect == 'sharpness':
        overall_mean_p.append(sharp[1])
        # plt.imsave(best_img_path, sharp[0])
    elif better_dect == 'contrast':
        overall_mean_p.append(contrast[1])
        # plt.imsave(best_img_path, contrast[0])
    elif better_dect == 'dark_sharp':
        overall_mean_p.append(dark_sharp[1])
        # plt.imsave(best_img_path, dark_sharp[0])
    elif better_dect == 'dark_bright':
        overall_mean_p.append(dark_bright[1])
        # plt.imsave(best_img_path, dark_bright[0])   
    elif better_dect == 'bright_sharp':
        overall_mean_p.append(bright_sharp[1])
        # plt.imsave(best_img_path, bright_sharp[0])
    elif better_dect == 'contrast_sharp':
        overall_mean_p.append(const_sharp[1])
        # plt.imsave(best_img_path, const_sharp[0])
    elif better_dect == 'bright_contrast':
        overall_mean_p.append(bright_const[1])
        # plt.imsave(best_img_path, bright_const[0])
    elif better_dect == 'color':
        overall_mean_p.append(color[1])
        # plt.imsave(best_img_path, color[0])
    elif better_dect == 'dehaze':
        overall_mean_p.append(dehaze[1])
        # plt.imsave(best_img_path, dehaze[0])

SyntaxError: EOL while scanning string literal (<ipython-input-9-33fea8ec5c0a>, line 3)

In [16]:
print('Overall Mean avg precision value {}'.format(sum(overall_mean_p)/len(overall_mean_p)))

Overall Mean avg precision value 0.9177902638912201
